# 🏥 HealthcareMAS Complete Workflow Function

Funzione unificata che incorpora tutto il pipeline medico-nutrizionale-psicologico con possibilità di aggiornamento tramite doctor_report.

In [1]:
def healthcare_mas_pipeline(
    clinical_summary: str,
    daily_data: dict,
    doctor_report: str = None,
    silent_mode: bool = True,
    save_folder: str = "medical_report"
) -> dict:
    """
    🏥 HealthcareMAS Complete Pipeline Function
    
    Esegue l'intero workflow: MedAI → NutriAI → PsyAI
    con possibilità di aggiornamento tramite doctor_report e salvataggio automatico
    
    Args:
        clinical_summary (str): Report clinico del paziente
        daily_data (dict): Dati giornalieri (biometrics + feedback)
        doctor_report (str, optional): Report del medico per aggiornamenti
        silent_mode (bool): Se True, sopprime output degli agenti
        save_folder (str): Nome della cartella dove salvare i report (default: "medical_report")
        
    Returns:
        dict: Risultato completo con tutti i piani e assessment
    """
    from agents.medico_base import get_medico
    from agents.nutrizionista import get_nutrizionista
    from agents.psicologo import get_psyai
    import json
    import os
    from datetime import datetime
    from pathlib import Path
    
    # Configura logging silenzioso se richiesto
    if silent_mode:
        os.environ["DATAPIZZA_AGENT_LOG_LEVEL"] = "ERROR"
    
    # Se presente doctor_report, aggiungilo al clinical_summary
    updated_clinical_summary = clinical_summary
    if doctor_report:
        updated_clinical_summary += f"\n\nDOCTOR REPORT (UPDATE):\n{doctor_report}"
    
    try:
        # 🩺 FASE 1: MedAI - Analisi medica e piano terapeutico
        print("🩺 Esecuzione MedAI - Analisi clinica...")
        medico = get_medico()
        medico_response = medico.run(updated_clinical_summary)
        piano_terapeutico = json.loads(medico_response.text)
        
        # 🥗 FASE 2: NutriAI - Piano nutrizionale sincronizzato
        print("🥗 Esecuzione NutriAI - Piano nutrizionale...")
        nutrizionista = get_nutrizionista()
        nutri_response = nutrizionista.run(json.dumps(piano_terapeutico))
        piano_settimanale = json.loads(nutri_response.text)
        
        # 🧠 FASE 3: PsyAI - Assessment psicologico e monitoring
        print("🧠 Esecuzione PsyAI - Assessment psicologico...")
        psicologo = get_psyai()
        
        input_psicologo = {
            "clinical_summary": updated_clinical_summary,
            "treatment_plan": piano_terapeutico,
            "weekly_meal_plan": piano_settimanale,
            **daily_data
        }
        
        psyai_response = psicologo.run(json.dumps(input_psicologo))
        psicologo_feedback = json.loads(psyai_response.text)
        
        # Estrai status e doctor_report
        pipeline_status = psicologo_feedback.get("pipeline_status", "UNKNOWN")
        new_doctor_report = psicologo_feedback.get("doctor_report")
        
        # Genera timestamp per i file
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Risultato completo
        result = {
            "timestamp": timestamp,
            "pipeline_status": pipeline_status,
            "medical_plan": piano_terapeutico,
            "nutrition_plan": piano_settimanale,
            "psychological_assessment": psicologo_feedback,
            "doctor_report_generated": new_doctor_report,
            "requires_medical_review": pipeline_status == "CRITICAL"
        }
        
        # 💾 SALVATAGGIO AUTOMATICO DEI FILE
        try:
            # Crea la cartella se non esiste
            save_path = Path(save_folder)
            save_path.mkdir(exist_ok=True)
            
            # 1. Piano terapeutico (JSON)
            medical_file = save_path / f"medical_plan_{timestamp}.json"
            with open(medical_file, 'w', encoding='utf-8') as f:
                json.dump(piano_terapeutico, f, indent=2, ensure_ascii=False)
            
            # 2. Piano nutrizionale (JSON)
            nutrition_file = save_path / f"nutrition_plan_{timestamp}.json"
            with open(nutrition_file, 'w', encoding='utf-8') as f:
                json.dump(piano_settimanale, f, indent=2, ensure_ascii=False)
            
            # 3. Assessment psicologico (JSON)
            psych_file = save_path / f"psychological_assessment_{timestamp}.json"
            with open(psych_file, 'w', encoding='utf-8') as f:
                json.dump(psicologo_feedback, f, indent=2, ensure_ascii=False)
            
            # 4. Summary completo (JSON)
            summary_file = save_path / f"complete_summary_{timestamp}.json"
            with open(summary_file, 'w', encoding='utf-8') as f:
                json.dump(result, f, indent=2, ensure_ascii=False)
            
            # 5. Report testuale leggibile (TXT)
            readable_file = save_path / f"human_readable_report_{timestamp}.txt"
            with open(readable_file, 'w', encoding='utf-8') as f:
                f.write(f"🏥 HEALTHCARE MAS REPORT - {timestamp}\n")
                f.write("=" * 60 + "\n\n")
                f.write(f"📊 PIPELINE STATUS: {pipeline_status}\n")
                f.write(f"⚕️  MEDICAL REVIEW: {'REQUIRED' if result['requires_medical_review'] else 'NOT REQUIRED'}\n\n")
                
                f.write("🩺 MEDICAL TREATMENT PLAN:\n")
                f.write("-" * 30 + "\n")
                f.write(f"Diagnosis: {piano_terapeutico.get('diagnostic_summary', 'N/A')}\n\n")
                
                for i, med in enumerate(piano_terapeutico.get('treatment_plan', []), 1):
                    f.write(f"{i}. {med.get('medication', 'N/A')} - {med.get('dosage', 'N/A')}\n")
                    f.write(f"   Frequency: {med.get('frequency', 'N/A')}\n")
                    f.write(f"   Instructions: {med.get('administration_instructions', 'N/A')}\n")
                    f.write(f"   Duration: {med.get('duration', 'N/A')}\n")
                    f.write(f"   Purpose: {med.get('purpose', 'N/A')}\n\n")
                
                f.write("🥗 NUTRITION PLAN:\n")
                f.write("-" * 30 + "\n")
                diet_info = piano_settimanale.get('diet_overview', {})
                f.write(f"Strategy: {diet_info.get('strategy_name', 'N/A')}\n")
                f.write(f"Daily Calories: {diet_info.get('daily_calorie_target', 'N/A')}\n")
                f.write(f"Macro Distribution: {diet_info.get('macro_distribution', 'N/A')}\n\n")
                
                f.write("🧠 PSYCHOLOGICAL ASSESSMENT:\n")
                f.write("-" * 30 + "\n")
                f.write(f"Analysis: {psicologo_feedback.get('daily_analysis', 'N/A')}\n")
                f.write(f"Adherence: {psicologo_feedback.get('adherence_score', 'N/A')}\n")
                f.write(f"Patient Message: {psicologo_feedback.get('patient_message', 'N/A')}\n")
                
                if psicologo_feedback.get('detected_anomalies'):
                    f.write(f"\n⚠️ DETECTED ANOMALIES:\n")
                    for anomaly in psicologo_feedback['detected_anomalies']:
                        f.write(f"- {anomaly}\n")
                
                if new_doctor_report and new_doctor_report != 'None':
                    f.write(f"\n🚨 DOCTOR REPORT:\n")
                    f.write(f"{new_doctor_report}\n")
            
            # 6. Lista della spesa (TXT) se disponibile
            if 'shopping_list_categories' in piano_settimanale:
                shopping_file = save_path / f"shopping_list_{timestamp}.txt"
                with open(shopping_file, 'w', encoding='utf-8') as f:
                    f.write(f"🛒 SHOPPING LIST - {timestamp}\n")
                    f.write("=" * 40 + "\n\n")
                    
                    for category, items in piano_settimanale['shopping_list_categories'].items():
                        f.write(f"📦 {category.upper()}:\n")
                        for item in items:
                            f.write(f"  □ {item}\n")
                        f.write("\n")
            
            print(f"💾 File salvati in: {save_path.absolute()}")
            print(f"   📄 {len(list(save_path.glob(f'*{timestamp}*')))} file generati")
            
            # Aggiungi percorsi dei file al result
            result["saved_files"] = {
                "folder": str(save_path.absolute()),
                "medical_plan": str(medical_file.absolute()),
                "nutrition_plan": str(nutrition_file.absolute()),
                "psychological_assessment": str(psych_file.absolute()),
                "complete_summary": str(summary_file.absolute()),
                "readable_report": str(readable_file.absolute())
            }
            
            if 'shopping_list_categories' in piano_settimanale:
                result["saved_files"]["shopping_list"] = str(shopping_file.absolute())
                
        except Exception as save_error:
            print(f"⚠️  Errore nel salvataggio: {save_error}")
            result["save_error"] = str(save_error)
        
        # Output finale
        print(f"✅ Pipeline completato - STATUS: {pipeline_status}")
        if pipeline_status == "CRITICAL":
            print(f"⚠️  ATTENZIONE: {new_doctor_report}")
        else:
            print("✅ Tutti i parametri sono nella norma")
            
        return result
        
    except Exception as e:
        print(f"❌ Errore nel pipeline: {str(e)}")
        return {
            "error": str(e),
            "pipeline_status": "ERROR",
            "requires_medical_review": True
        }

## 📋 Esempio di utilizzo

In [ ]:
from agents.medico_base import clinical_summary
from agents.psicologo import daily_payload

# Esecuzione del pipeline completo con salvataggio automatico
result = healthcare_mas_pipeline(
    clinical_summary=clinical_summary,
    daily_data=daily_payload,
    doctor_report=None,  # Nessun aggiornamento medico
    silent_mode=True,
    save_folder="medical_reports_test"  # Cartella personalizzata
)

🩺 Esecuzione MedAI - Analisi clinica...
🥗 Esecuzione NutriAI - Piano nutrizionale...
🥗 Esecuzione NutriAI - Piano nutrizionale...
🧠 Esecuzione PsyAI - Assessment psicologico...
🧠 Esecuzione PsyAI - Assessment psicologico...
💾 File salvati in: f:\HealthcareMAS\medical_reports_test
   📄 6 file generati
✅ Pipeline completato - STATUS: CRITICAL
⚠️  ATTENZIONE: The low sleep hours and HRV suggest physiological stress, potentially exacerbating hypertension and diabetes. Confirmed non-adherence to medication needs to be addressed urgently to avoid further health issues.
💾 File salvati in: f:\HealthcareMAS\medical_reports_test
   📄 6 file generati
✅ Pipeline completato - STATUS: CRITICAL
⚠️  ATTENZIONE: The low sleep hours and HRV suggest physiological stress, potentially exacerbating hypertension and diabetes. Confirmed non-adherence to medication needs to be addressed urgently to avoid further health issues.


In [3]:
# Visualizza il risultato strutturato
print("📊 RISULTATI DEL PIPELINE:")
print("=" * 50)
print(f"Status: {result['pipeline_status']}")
print(f"Revisione medica richiesta: {result['requires_medical_review']}")
print(f"Timestamp: {result['timestamp']}")

if result.get('doctor_report_generated') and result['doctor_report_generated'] != 'None':
    print(f"\n⚠️ Doctor Report: {result['doctor_report_generated']}")

# Accesso ai singoli piani
medical_plan = result['medical_plan']
nutrition_plan = result['nutrition_plan']
psych_assessment = result['psychological_assessment']

print(f"\n🩺 Piano medico generato con {len(medical_plan['treatment_plan'])} farmaci")
print(f"🥗 Piano nutrizionale per 7 giorni")
print(f"🧠 Assessment psicologico: {psych_assessment.get('adherence_score', 'N/A')}")

# Mostra informazioni sui file salvati
if "saved_files" in result:
    print(f"\n💾 FILE SALVATI:")
    saved_files = result["saved_files"]
    print(f"📁 Cartella: {saved_files['folder']}")
    print(f"📄 Piano medico: {saved_files['medical_plan']}")
    print(f"🥗 Piano nutrizionale: {saved_files['nutrition_plan']}")
    print(f"🧠 Assessment psicologico: {saved_files['psychological_assessment']}")
    print(f"📋 Report completo: {saved_files['complete_summary']}")
    print(f"📖 Report leggibile: {saved_files['readable_report']}")
    if "shopping_list" in saved_files:
        print(f"🛒 Lista spesa: {saved_files['shopping_list']}")

📊 RISULTATI DEL PIPELINE:
Status: CRITICAL
Revisione medica richiesta: True
Timestamp: 20251122_160022

⚠️ Doctor Report: The low sleep hours and HRV suggest physiological stress, potentially exacerbating hypertension and diabetes. Confirmed non-adherence to medication needs to be addressed urgently to avoid further health issues.

🩺 Piano medico generato con 4 farmaci
🥗 Piano nutrizionale per 7 giorni
🧠 Assessment psicologico: LOW

💾 FILE SALVATI:
📁 Cartella: f:\HealthcareMAS\medical_reports_test
📄 Piano medico: f:\HealthcareMAS\medical_reports_test\medical_plan_20251122_160022.json
🥗 Piano nutrizionale: f:\HealthcareMAS\medical_reports_test\nutrition_plan_20251122_160022.json
🧠 Assessment psicologico: f:\HealthcareMAS\medical_reports_test\psychological_assessment_20251122_160022.json
📋 Report completo: f:\HealthcareMAS\medical_reports_test\complete_summary_20251122_160022.json
📖 Report leggibile: f:\HealthcareMAS\medical_reports_test\human_readable_report_20251122_160022.txt
🛒 Lista 

## 🔄 Esempio con Doctor Report (aggiornamento)

Simuliamo un caso in cui il medico fornisce un aggiornamento al piano terapeutico

In [ ]:
# Simulazione di un doctor report per aggiornamento terapia
doctor_update = """
MEDICAL UPDATE - Dr. Smith Review (2025-11-22):

After reviewing the initial treatment plan and patient's response:
1. Metformin dosage should be increased to 1000mg twice daily due to persistent HbA1c levels
2. Add Atorvastatin 20mg daily for cholesterol management 
3. Patient reports good tolerance to current medications
4. Continue current GERD treatment but monitor for effectiveness
5. Schedule follow-up in 2 weeks to assess glucose control

PRIORITY: Monitor for any signs of lactic acidosis with increased Metformin dose.
"""

if result["requires_medical_review"]:
    print("\n⚠️ Il pipeline richiede una revisione medica basata sui dati forniti.")
    doctor_update = result["doctor_report_generated"]

# Nuova esecuzione con doctor_report e cartella separata
from agents.psicologo import daily_payload  # Usiamo scenario più critico

result_updated = healthcare_mas_pipeline(
    clinical_summary=clinical_summary,
    daily_data=daily_payload,  # Dati meno ottimali
    doctor_report=doctor_update,  # Aggiornamento del medico
    silent_mode=True,
    save_folder="medical_reports_updated"  # Cartella separata per aggiornamenti
)


⚠️ Il pipeline richiede una revisione medica basata sui dati forniti.
🩺 Esecuzione MedAI - Analisi clinica...
🥗 Esecuzione NutriAI - Piano nutrizionale...
🥗 Esecuzione NutriAI - Piano nutrizionale...
🧠 Esecuzione PsyAI - Assessment psicologico...
🧠 Esecuzione PsyAI - Assessment psicologico...
💾 File salvati in: f:\HealthcareMAS\medical_reports_updated
   📄 6 file generati
✅ Pipeline completato - STATUS: CRITICAL
⚠️  ATTENZIONE: Patient skipped Metformin doses which is critical for managing Type 2 Diabetes. Low HRV and insufficient sleep indicate high physiological stress, exacerbating health issues. Immediate intervention is necessary.
💾 File salvati in: f:\HealthcareMAS\medical_reports_updated
   📄 6 file generati
✅ Pipeline completato - STATUS: CRITICAL
⚠️  ATTENZIONE: Patient skipped Metformin doses which is critical for managing Type 2 Diabetes. Low HRV and insufficient sleep indicate high physiological stress, exacerbating health issues. Immediate intervention is necessary.


In [5]:
# Confronto dei risultati
print("🔍 CONFRONTO RISULTATI:")
print("=" * 60)

print(f"📊 ESECUZIONE ORIGINALE:")
print(f"   Status: {result['pipeline_status']}")
print(f"   Farmaci prescritti: {len(result['medical_plan']['treatment_plan'])}")
print(f"   File salvati in: {result.get('saved_files', {}).get('folder', 'N/A')}")

print(f"\n📊 ESECUZIONE CON DOCTOR REPORT:")
print(f"   Status: {result_updated['pipeline_status']}")
print(f"   Farmaci prescritti: {len(result_updated['medical_plan']['treatment_plan'])}")
print(f"   File salvati in: {result_updated.get('saved_files', {}).get('folder', 'N/A')}")

# Mostra differenze nei farmaci
print(f"\n💊 CONFRONTO TERAPIE:")
for i, med in enumerate(result_updated['medical_plan']['treatment_plan']):
    print(f"   {i+1}. {med['medication']} - {med['dosage']} ({med['frequency']})")

if result_updated.get('doctor_report_generated') != 'None':
    print(f"\n⚠️ NUOVO DOCTOR REPORT GENERATO:")
    print(f"   {result_updated.get('doctor_report_generated')}")

# Mostra statistiche sui file salvati
print(f"\n📁 GESTIONE FILE:")
if "saved_files" in result:
    original_files = len([f for f in result["saved_files"].values() if isinstance(f, str) and f.endswith(('.json', '.txt'))])
    print(f"   Report originale: {original_files} file")

if "saved_files" in result_updated:
    updated_files = len([f for f in result_updated["saved_files"].values() if isinstance(f, str) and f.endswith(('.json', '.txt'))])
    print(f"   Report aggiornato: {updated_files} file")

🔍 CONFRONTO RISULTATI:
📊 ESECUZIONE ORIGINALE:
   Status: CRITICAL
   Farmaci prescritti: 4
   File salvati in: f:\HealthcareMAS\medical_reports_test

📊 ESECUZIONE CON DOCTOR REPORT:
   Status: CRITICAL
   Farmaci prescritti: 4
   File salvati in: f:\HealthcareMAS\medical_reports_updated

💊 CONFRONTO TERAPIE:
   1. Metformin - 500mg (1 tablet twice daily)
   2. Lisinopril - 10mg (1 tablet once daily)
   3. Atorvastatin - 20mg (1 tablet once daily)
   4. Omeprazole - 20mg (1 capsule once daily)

⚠️ NUOVO DOCTOR REPORT GENERATO:
   Patient skipped Metformin doses which is critical for managing Type 2 Diabetes. Low HRV and insufficient sleep indicate high physiological stress, exacerbating health issues. Immediate intervention is necessary.

📁 GESTIONE FILE:
   Report originale: 6 file
   Report aggiornato: 6 file
